In [3]:
import pandas as pd

In [4]:
import wandb
run = wandb.init()
artifact = run.use_artifact('parambharat/wandbot_eval/eval_results:v1', type='dataset')
artifact_dir = artifact.download()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: capecape. Use `wandb login --relogin` to force relogin


wandb:   1 of 1 files downloaded.  


In [5]:
table = artifact.get("eval_results")

wandb:   1 of 1 files downloaded.  


In [6]:
table.columns

['question',
 'answer',
 'reaction',
 'contexts',
 'answer_relevancy',
 'context_relavency',
 'faithfulness',
 'correctness']

In [7]:
df = pd.DataFrame(table.data, columns=table.columns)

In [22]:
df = df[df.reaction!=-1]

In [23]:
df.head()

,question,answer,reaction,contexts,answer_relevancy,context_relavency,faithfulness,correctness
0,"write sample code to upload a table to a run, ...","Sure, here is how you can upload a table to a ...",0,[description: Explore how to use W&B Tables wi...,0.897,0.0,0.333333,1
1,"write sample code to upload a table to a run, ...","Sure, here is how you can upload a table to a ...",0,[description: How to use the sidebar and table...,0.884,0.0,1.000000,1
2,is there an integration with MONAI?,"Hmm, I'm not sure. The provided documentation ...",0,[Integrations\n\nView source on GitHub\n\nModu...,0.884,0.0,0.000000,1
4,how do I resume a run and attach an artifact t...,**Warning: Falling back to gpt-3.5.** These re...,0,[```\n\nResume Guidance\n\nThere are different...,0.911,0.0,0.500000,1
5,can i attach an artifact to an already existin...,"Yes, you can attach an artifact to an already ...",0,[```\n\n✔️ run.use_artifact\n\nThese steps are...,0.850,0.0,0.000000,1


In [24]:
df.iloc[0]["question"]

'write sample code to upload a table to a run, then download that table again in a new run as a Pandas Dataframe'

In [25]:
df.iloc[0]["answer"]

'Sure, here is how you can upload a table to a run and then download it in a new run as a Pandas DataFrame:\n\nFirst, let\'s create a table and log it to a run:\n\n```python\nimport wandb\n\n# Initialize a new run\nrun = wandb.init(project="table-test")\nmy_table = wandb.Table(\n    columns=["a", "b"], \n    data=[["a1", "b1"], ["a2", "b2"]]\n    )\nrun.log({"Table Name": my_table})\n```\n\nNow, let\'s download the table in a new run:\n\n```python\nimport wandb\n\n# Initialize a new run\nrun = wandb.init(project="table-test")\n\n# Use the artifact API to access the table\nartifact = run.use_artifact("run-<run-id>-Table Name:<tag>")\ntable = artifact.get("Table Name")\n\n# Convert the table to a Pandas DataFrame\ndf = table.to_pandas()\n```\n\nPlease replace `<run-id>` and `<tag>` with the actual run ID and tag. The run ID can be found in the URL of the run page in the Weights & Biases web interface. The tag is the version of the artifact, which is usually a number.\n\nFor more informat

remove bad reactions (score from users input)

In [26]:
row = df.iloc[0].to_dict()

In [33]:
# Bharat's super prompt
{
  "system_template": "You are an AI assistant designed to assist developers with everyday tasks related to Weight & Biases and provide helpful information. As an expert in the open-source python SDK wandb. Here are your guidelines:\n1. Answer user questions only using the provided document excerpts from the official Weight & Biases [documentation](https://docs.wandb.ai) and code examples and not prior knowledge.\n2. Provide clear and concise explanations, along with relevant code snippets, to help users understand and instrument various functionalities of wandb efficiently.\n3. You must cite the source from the provided context while generating your response.\n4. Where the provided context is insufficient and you are uncertain about the response, respond with \"Hmm, I'm not sure.\"  and direct the user to the weights & biases [support](support@wandb.com) or forums(http://wandb.me/community)\n5. For questions unrelated to wandb or Weights & Biases, kindly remind the user of your specialization\n6. Always respond in fully formatted markdown and in the language of the user's question.\n<!--start-example1-input-->\nHuman: How do I log audio using wandb?\n================\nContent: Weights & Biases allows logging of audio data arrays or files for playback in W&B. Use `wandb.Audio()` to log audio.\nSource: 28-pl\nContent: # Log an audio array or file\nwandb.log({{\"my whale song\": wandb.Audio(array_or_path, caption=\"montery whale 0034\", sample_rate=32)}})\n\n# OR\n\n# Log audio within a W&B Table\nmy_table = wandb.Table(columns=[\"audio\", \"spectrogram\", \"bird_class\", \"prediction\"])\nfor (audio_arr, spec, label) in my_data:\n    pred = model(audio)\n    audio = wandb.Audio(audio_arr, sample_rate=32)\n    img = wandb.Image(spec)\n    my_table.add_data(audio, img, label, pred)\n\n# Log the Table to wandb\nwandb.log({{\"validation_samples\" : my_table}})\nSource: 30-pl,\n================\nFinal Answer in Markdown:\n<!--end-example1-input-->\n<!--start-example1-output-->\nTo log audio with wandb, follow this example:\n\n```\nimport wandb\n\naudio = wandb.data_types.Audio(data_or_path=\"path/to/audio.wav\", sample_rate=44100, caption=\"My audio clip\")\ndurations = audio.durations()\nsample_rates = audio.sample_rates()\nwandb.log({{\"audio\": audio}})\n```\nSources: 30-pl\n<!--end-example1-output-->\n<!--start-example2-input-->\nHuman: How to eat vegetables using pandas?\n================\nContent: ExtensionArray.repeat(repeats, axis=None) is a method to repeat elements of an ExtensionArray. Parameters include repeats (int or array of ints) and axis (0 or 'index', 1 or 'columns'), with axis=0 being the default.\nSource: 0-pl\n================\nFinal Answer in Markdown:\n<!--end-example2-input-->\n<!--start-example2-output-->\nYour question doesn't pertain to wandb. I'm here to assist with wandb-related queries. Please ask a wandb-specific question\nSources: \n<!--end-example2-output-->",
  "human_template": "<!--start-input-->\n\n\n{question}\n================\n{summaries}\n================\nFinal Answer in Markdown:\"\n<!--end-input-->\n<!--start-output-->"
}

system_prompt = "You are an AI assistant designed to assist developers with everyday tasks related to Weight & Biases and provide helpful information. As an expert in the open-source python SDK wandb answer the following question based on the context."

In [39]:
def prompt(row):
    p = """Context: {context}

Question: {question}

""".format(context="-".join(row["contexts"]), question=row["question"])

    return {
        "messages": [
            { "role": "system", "content": system_prompt },
            { "role": "user", "content": p },
            { "role": "assistant", "content": row["answer"] }
        ]
        }

In [43]:
df.apply(prompt, axis=1).to_json("ft.jsonl", orient="records", lines=True)

In [44]:
# We start by importing the required packages

import json
import os
import tiktoken
import numpy as np
from collections import defaultdict

# Next, we specify the data path and open the JSONL file

data_path = "ft.jsonl"

# Load dataset
with open(data_path) as f:
    dataset = [json.loads(line) for line in f]

# We can inspect the data quickly by checking the number of examples and the first item

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)

# Now that we have a sense of the data, we need to go through all the different examples and check to make sure the formatting is correct and matches the Chat completions message structure

# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue

    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue

    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1

        if any(k not in ("role", "content", "name") for k in message):
            format_errors["message_unrecognized_key"] += 1

        if message.get("role", None) not in ("system", "user", "assistant"):
            format_errors["unrecognized_role"] += 1

        content = message.get("content", None)
        if not content or not isinstance(content, str):
            format_errors["missing_content"] += 1

    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

# Beyond the structure of the message, we also need to ensure that the length does not exceed the 4096 token limit.

# Token counting functions
encoding = tiktoken.get_encoding("cl100k_base")

# not exact!
# simplified from https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

# Last, we can look at the results of the different formatting operations before proceeding with creating a fine-tuning job:

# Warnings and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))

print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
n_too_long = sum(l > 4096 for l in convo_lens)
print(f"\n{n_too_long} examples may be over the 4096 token limit, they will be truncated during fine-tuning")

# Pricing and default n_epochs estimate
MAX_TOKENS_PER_EXAMPLE = 4096

MIN_TARGET_EXAMPLES = 100
MAX_TARGET_EXAMPLES = 25000
TARGET_EPOCHS = 3
MIN_EPOCHS = 1
MAX_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")
print("See pricing page to estimate total costs")

Num examples: 811
First example:
{'role': 'system', 'content': 'You are an AI assistant designed to assist developers with everyday tasks related to Weight & Biases and provide helpful information. As an expert in the open-source python SDK wandb answer the following question based on the context.'}
{'role': 'user', 'content': 'Context: description: Explore how to use W&B Tables with this 5 minute Quickstart.\ndisplayed_sidebar: default\n\nimport Tabs from \'@theme/Tabs\';\nimport TabItem from \'@theme/TabItem\';\n\nWalkthrough\n\nThe following Quickstart demonstrates how to log data tables, visualize data, and query data.\n\nSelect the button below to try a PyTorch Quickstart example project on MNIST data.\n\n1. Log a table\n\nLog a table with W&B. You can either construct a new table or pass a Pandas DataFrame.\n\nwandb.init(): Create a\n\nrun to track results.\n-\n\nwandb.Table(): Create a new table object.\n  -\n\nrun.log(): Log the table to save it to W&B.\n\nHere\'s an example:\n

## FT 3.5

In [1]:
import openai

In [8]:
f = openai.File.create(
  file=open("ft.jsonl", "rb"),
  purpose='fine-tune'
)

In [17]:
# wait to the file to be ready
openai.File.list()

<OpenAIObject list at 0x137b26070> JSON: {
  "object": "list",
  "data": [
    {
      "object": "file",
      "id": "file-fEJimRRd32E2gKw8ptsHF3Dd",
      "purpose": "fine-tune",
      "filename": "./artifacts/wiki-dataset-train:v1/wiki_train.jsonl",
      "bytes": 9443981,
      "created_at": 1633617202,
      "status": "processed",
      "status_details": null
    },
    {
      "object": "file",
      "id": "file-fs3HurfmvohWyvLdsU6rM09n",
      "purpose": "fine-tune-results",
      "filename": "compiled_results.csv",
      "bytes": 57934,
      "created_at": 1633618849,
      "status": "processed",
      "status_details": null
    },
    {
      "object": "file",
      "id": "file-NeSjcChN0nhD2FwOJuqVxRxh",
      "purpose": "fine-tune-results",
      "filename": "compiled_results.csv",
      "bytes": 58077,
      "created_at": 1633625775,
      "status": "processed",
      "status_details": null
    },
    {
      "object": "file",
      "id": "file-gikREx5ssHFGVyUKxFn5wixG",
    

In [21]:
# openai.api_key = openai_api_key
openai.FineTuningJob.create(
    training_file=f.id,
    model="gpt-3.5-turbo"
)

<FineTuningJob fine_tuning.job id=ftjob-t3SG14m9WW3oO5BACIgDzIG2 at 0x127681b20> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-t3SG14m9WW3oO5BACIgDzIG2",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1693219705,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-WnF2wEqNkV1Nj65CzDxr6iUm",
  "result_files": [],
  "status": "created",
  "validation_file": null,
  "training_file": "file-RNM7MQZQxBh1loxfI6oZf2hF",
  "hyperparameters": {
    "n_epochs": 3
  },
  "trained_tokens": null
}

In [22]:
openai.FineTuningJob.list(limit=10)

<OpenAIObject list at 0x1400618f0> JSON: {
  "object": "list",
  "data": [
    {
      "object": "fine_tuning.job",
      "id": "ftjob-t3SG14m9WW3oO5BACIgDzIG2",
      "model": "gpt-3.5-turbo-0613",
      "created_at": 1693219705,
      "finished_at": null,
      "fine_tuned_model": null,
      "organization_id": "org-WnF2wEqNkV1Nj65CzDxr6iUm",
      "result_files": [],
      "status": "running",
      "validation_file": null,
      "training_file": "file-RNM7MQZQxBh1loxfI6oZf2hF",
      "hyperparameters": {
        "n_epochs": 3
      },
      "trained_tokens": null
    },
    {
      "object": "fine_tuning.job",
      "id": "ftjob-fGhc0eY9du8912QSel5wA1fG",
      "model": "gpt-3.5-turbo-0613",
      "created_at": 1693170316,
      "finished_at": 1693174227,
      "fine_tuned_model": "ft:gpt-3.5-turbo-0613:weights-biases::7sIFcDXl",
      "organization_id": "org-WnF2wEqNkV1Nj65CzDxr6iUm",
      "result_files": [
        "file-925sFoFII12KSQg9XG6uQGOI"
      ],
      "status": "succeed

In [37]:
state = openai.FineTuningJob.retrieve("ftjob-t3SG14m9WW3oO5BACIgDzIG2")
state["status"], state["trained_tokens"], state["finished_at"]

('succeeded', 9435441, 1693224539)

In [38]:
state

<FineTuningJob fine_tuning.job id=ftjob-t3SG14m9WW3oO5BACIgDzIG2 at 0x13044db70> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-t3SG14m9WW3oO5BACIgDzIG2",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1693219705,
  "finished_at": 1693224539,
  "fine_tuned_model": "ft:gpt-3.5-turbo-0613:weights-biases::7sVL6MDK",
  "organization_id": "org-WnF2wEqNkV1Nj65CzDxr6iUm",
  "result_files": [
    "file-FLrzyMgz8aVFUuzEx7QXPyCY"
  ],
  "status": "succeeded",
  "validation_file": null,
  "training_file": "file-RNM7MQZQxBh1loxfI6oZf2hF",
  "hyperparameters": {
    "n_epochs": 3
  },
  "trained_tokens": 9435441
}

In [33]:
!openai wandb sync --project "wandbot_ft" --id "ftjob-t3SG14m9WW3oO5BACIgDzIG2"

Retrieving fine-tune job...
wandb: Currently logged in as: capecape. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.15.8 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.15.3
wandb: Run data is saved locally in /Users/tcapelle/work/openai_ft/artifacts/eval_results:v1/wandb/run-20230828_144744-ftjob-t3SG14m9WW3oO5BACIgDzIG2
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run ftjob-t3SG14m9WW3oO5BACIgDzIG2
wandb: ⭐️ View project at https://wandb.ai/capecape/wandbot_ft
wandb: 🚀 View run at https://wandb.ai/capecape/wandbot_ft/runs/ftjob-t3SG14m9WW3oO5BACIgDzIG2
wandb: Waiting for W&B process to finish... (success).
wandb: 
wandb: Run history:
wandb: train_accuracy █▁▁█▁▁▁██▁█▁███▁▁██▁██▁██▁█▁▁▁█▁████████
wandb:     train_loss ▁▄▄▁▄▃▃▁▁▄▁▂▁▁▁▃▂▁▁▄▁▁▂▁▁▃▁▂▃▁▁█▁▁▁▁▁▁▁▁
wandb: 
wandb: Run summary:
wandb: fine_tuned_model ft:gpt-3.5-turbo-061...
wandb:           status succeeded
wandb:   tr

## Use the model

In [49]:
completion = openai.ChatCompletion.create(
  model=state["fine_tuned_model"],
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "How do I launch a job on wandb?"}
  ]
)

In [50]:
print(completion["choices"][0]["message"]["content"])

 To launch a job on Weights & Biases (wandb), follow these steps:

1. Initialize wandb in your script:

```python
import wandb
wandb.init()
```

2. Define your hyperparameters and settings with the `wandb.config` object:

```python
wandb.config.learning_rate = 0.001
wandb.config.dropout = 0.2
```

3. After training your model, log the relevant metrics and artifacts:

```python
accuracy, loss = evaluate_model(...)
wandb.log({"accuracy": accuracy, "loss": loss})
wandb.save('model.h5')
```

4. To launch a sweep:

```python
sweep_id = wandb.sweep(sweep_config)
wandb.agent(sweep_id, function=train)
```

For more detailed information, check the [official wandb documentation](https://docs.wandb.ai/).
